# HW1
## Jie Liu, Xuening Li, Jiamin Zhu, Lemeng zhou

In [31]:
import numpy as np
import pandas as pd
from scipy import stats                 
from scipy.stats import norm
import datetime
import math

In [32]:
pd.options.display.max_columns = 10     # sets viewing configuration for easier readability in the console

In [33]:
# ========== FUNCTION DEFINITIONS ==========
def proportion(array_TF):
    return sum(array_TF)/len(array_TF)

In [34]:
def calc_zscore(phat, p_f, n_f):
    z_score_f = (phat-p_f)/((p_f*(1-p_f))/n_f)**0.5     ##### Replace None with formula
    return z_score_f


In [35]:
def get_z_crit_value(alpha_f, num_sides_f):
    z_crit_value_f = norm.ppf(1-(alpha_f/num_sides_f))   ##### Replace None with formula; hint: use norm.ppf package
    return z_crit_value_f

In [36]:
def get_p_value(zscore_f, num_sides_f):
    ### p_value_f = (1-norm.cdf(zscore_f))/num_sides_f
    p_value_f = num_sides_f * min(norm.cdf(zscore_f), 1 - norm.cdf(zscore_f)) ##### Replace None with formula; hint: use norm.cdf package
    return p_value_f

In [37]:
def reject_null(variantA_outcomes_f, variantB_outcomes_f, alpha_f, num_sides_f):
    p_hat_f = proportion(variantB_outcomes_f)
    p_f = proportion(variantA_outcomes_f)
    n_f = len(variantB_outcomes_f)
    z_score = calc_zscore(p_hat_f, p_f, n_f)
    p_value = get_p_value(z_score, num_sides_f)
    z_crit = get_z_crit_value(alpha_f, num_sides_f)
    
    ##### reject_null_TF_f = None         
    ##### Replace None with formula. This should result in a boolean variable (True or False). You can check the variable type in the console with the command: "type(reject_null_TF_f)"
    if z_score >= z_crit and p_value < alpha_f:
    # if z_score >= z_crit:
        reject_null_TF_f = True
    else:
        reject_null_TF_f = False

    return reject_null_TF_f, z_score, p_value

In [38]:
def calc_optimal_sample_size(p0_f, mde_f, alpha_f, power_f):
    t_alpha2 = abs(norm.ppf(alpha_f/2))
    t_beta = abs(norm.ppf(1-power_f))
    p1_f = p0_f + mde_f
    p_avg = (p0_f + (p0_f + mde_f))/2
    sample_size = pow((t_alpha2*(pow(2*p_avg*(1-p_avg), 0.5)) + t_beta*pow((p0_f*(1-p0_f)+p1_f*(1-p1_f)), 0.5)), 2)*(1/pow(mde_f, 2)) ##### Replace None with formula
    return sample_size


In [39]:
#DECLARE PARAMETERS


trial_start_date=datetime.date(year=2020, month=8, day=1)

#LOAD DATA
df = pd.read_csv('AB_test_data.csv')
df.date = pd.to_datetime(df.date, format='%Y-%m-%d')    # parse string format
df.date = df.date.apply(lambda x: datetime.date(year=x.year, month=x.month, day=x.day)) # convert to standard (non-pandas) format for comparison against other dates


In [40]:
df.head()

,purchase_TF,Variant,date,id
0,False,A,2019-11-08,0x25b44a
1,False,B,2020-08-27,0x46271e
2,False,A,2020-06-11,0x80b8f1
3,False,B,2020-08-22,0x8d736d
4,False,A,2020-08-05,0x96c9c8


In [41]:
# ========== ANALYSES ==========
# ----- Get summary stats -----
df['year'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month
df_summary = df[['year', 'month', 'Variant', 'id', 'purchase_TF']].groupby(['year', 'month', 'Variant']).agg({'id': 'count', 'purchase_TF': 'sum'}).rename(columns={'id': 'num_exposures', 'purchase_TF': 'num_bookings'})
df_summary['conv_rate'] = df_summary['num_bookings']/df_summary['num_exposures']
perc_vA = df_summary.loc[(trial_start_date.year, trial_start_date.month, 'A'), 'num_bookings'] / df_summary.loc[(trial_start_date.year, trial_start_date.month, 'A'), 'num_exposures']
perc_vB = df_summary.loc[(trial_start_date.year, trial_start_date.month, 'B'), 'num_bookings'] / df_summary.loc[(trial_start_date.year, trial_start_date.month, 'B'), 'num_exposures']


In [42]:
df_summary

num_exposures  num_bookings  conv_rate
year month Variant                                        
2019 8     A                10176          1523   0.149666
     9     A                 9810          1498   0.152701
     10    A                10179          1532   0.150506
     11    A                 9957          1398   0.140404
     12    A                10249          1493   0.145673
2020 1     A                10248          1558   0.152030
     2     A                 9446          1387   0.146835
     3     A                10021          1488   0.148488
     4     A                 9780          1539   0.157362
     5     A                10162          1538   0.151348
     6     A                 9856          1460   0.148133
     7     A                10116          1514   0.149664
     8     A                 5000           774   0.154800
           B                 5000           883   0.176600

In [43]:
print('For month beginning %s, Variant A had %d exposures (%3.1f%%) and Variant B had %d exposures (%3.1f%%)' % (trial_start_date, int(df_summary.loc[(trial_start_date.year, trial_start_date.month, 'A'), 'num_exposures']), perc_vA*100, int(df_summary.loc[(trial_start_date.year, trial_start_date.month, 'B'), 'num_exposures']), perc_vB*100))

For month beginning 2020-08-01, Variant A had 5000 exposures (15.5%) and Variant B had 5000 exposures (17.7%)


In [44]:
# Question 1
# set parameters
alpha = 0.05    # significance level
num_sides = 1   # one-sided=1 or two-sided=2 test


# ALL DATA
variantA_outcomes = df.loc[df['Variant'] == 'A', 'purchase_TF']
variantB_outcomes = df.loc[df['Variant'] == 'B', 'purchase_TF']

#conduct tests
reject_null_test, z_score, p_value = reject_null(variantA_outcomes, variantB_outcomes, alpha, num_sides)
print('Conversion rate for Variant A: %3.1f%%' % (proportion(variantA_outcomes)*100))
print('Conversion rate for Variant B: %3.1f%%' % (proportion(variantB_outcomes)*100))
print('Using all Variant B, reject null T/F?: %s' % reject_null_test)
print('z-score = %3.2f and p-value = %3.9f%%' % (z_score, p_value*100))

Conversion rate for Variant A: 15.0%
Conversion rate for Variant B: 17.7%
Using all Variant B, reject null T/F?: True
z-score = 5.35 and p-value = 0.000004415%


In [45]:
# ----- Question 2 -----
# set parameters
alpha = 0.05    # significance level
power = 0.80    # power of test
mde = 0.03      # desired minimum detectable effect
num_sides = 1   # one-sided or two-sided test

# --- Configure data
variantA_outcomes_control = df.loc[(df['Variant'] == 'A') & (df.date < trial_start_date), 'purchase_TF']
variantA_outcomes = df.loc[(df['Variant'] == 'A') & (df.date >= trial_start_date), 'purchase_TF']
variantB_outcomes = df.loc[(df['Variant'] == 'B') & (df.date >= trial_start_date), 'purchase_TF']

# --- Calculate and display the optimal sample size
p0 = proportion(variantA_outcomes_control)     # this would be the baseline prior to starting a test since we would only have historical information
n_star = int(np.ceil(calc_optimal_sample_size(p0, mde, alpha, power)))    # optimal sample size; rounding up using np.ceil because sample must be a whole number that is *at least* as large as the optimal size
num_samples = 10


print('The optimal sample size is %d ' % n_star)
if n_star > variantB_outcomes.shape[0]:
    print('Warning: optimal sample size is larger than number of observations.')

# --- conduct test for n samples of the optimal size
# initialize list to store variables from each loop iteration
variantB_outcomes_samples = list()  # this will store the data samples
reject_null_list = list()   # this will store the results of the significance tests
z_score_list = list()   # this still store the z-scores from each test
p_value_list = list()   # this will store the p-values from each test
for i in range(0, num_samples):
    t_perc_of_trial_data_to_use = n_star/variantB_outcomes.shape[0]  ##### Replace None with the formula for what percent of the trial data should be used
    t_sample = variantB_outcomes.sample(frac=min(t_perc_of_trial_data_to_use, 1))   ##### No changes needed, but think about why we're using a min() function here
    t_reject, t_z_score, t_p_value = reject_null(variantA_outcomes, t_sample, alpha, num_sides)
    variantB_outcomes_samples.append(list(t_sample))    ##### No changes needed, but investigate what the append function is doing here
    reject_null_list.append(t_reject)
    z_score_list.append(t_z_score)
    p_value_list.append(t_p_value)
    

print("For %d samples of optimal sample size %d, %3.2f%% rejected the null" % (num_samples, n_star, proportion(reject_null_list)*100))

df_sample_summary = pd.DataFrame(data={'sample number': list(range(0, num_samples)), 'z_score': z_score_list, 'p_value': p_value_list})
print(df_sample_summary[['sample number', 'z_score', 'p_value']])
df_sample_summary.to_csv('sample_summary.csv', index=False)

The optimal sample size is 2395 
For 10 samples of optimal sample size 2395, 100.00% rejected the null
   sample number   z_score   p_value
0              0  2.048038  0.020278
1              1  3.121374  0.000900
2              2  3.516813  0.000218
3              3  3.516813  0.000218
4              4  3.686287  0.000114
5              5  2.330495  0.009890
6              6  3.403830  0.000332
7              7  4.138218  0.000018
8              8  3.686287  0.000114
9              9  3.799270  0.000073


In [46]:
###Question 3

In [47]:
# set parameters
upp_bdr = math.log(1/alpha) 
lower_bdr = math.log(1-power)

In [50]:
i = 0
ln_val = 0
cnt = 0 ## how many samples have been tested
num = 0 ## sum of trails in each test

#conduct 10 sequential tests 
for i in range(10):
    ln_lambda = 0
    j = 0
    A_data = df[df['Variant']=='A'].sample(n=n_star, random_state=i*1000, axis=None)
    mean_A = np.mean(A_data['purchase_TF'])
    B_data = df[df['Variant']=='B'].sample(n=n_star, random_state=i*1000, axis=None)
    mean_B = np.mean(B_data['purchase_TF'])
    for x in range(2400):
        d = int(B_data['purchase_TF'][j:j+1]) 
        if d == 1:
            ln_val = math.log(mean_B/mean_A)
        elif d < 1:
            ln_val = math.log((1-mean_B)/(1-mean_A))
        ln_lambda += ln_val
        if ln_lambda <= lower_bdr:
            print('Sample ',i+1,', trail' ,j ,', not rejecting H0')
            print('ln lambda is', ln_lambda)
            cnt += j
            break
        elif ln_lambda >= upp_bdr:
            print('Sample ',i+1,', trail' ,j ,', rejecting H0')
            print('ln lambda is',ln_lambda)
            cnt += j
            break
            
        j += 1
    num += j
    i += 1

Sample  1 , trail 402 , rejecting H0
ln lambda is 3.0237460723853524
Sample  2 , trail 641 , rejecting H0
ln lambda is 3.050257452502628
Sample  3 , trail 558 , rejecting H0
ln lambda is 3.001960094307472
Sample  4 , trail 64 , not rejecting H0
ln lambda is -1.650732726708318
Sample  5 , trail 2174 , rejecting H0
ln lambda is 3.062218249010228
Sample  6 , trail 125 , rejecting H0
ln lambda is 3.0855494812917117
Sample  7 , trail 352 , rejecting H0
ln lambda is 3.0843246663171344
Sample  8 , trail 348 , rejecting H0
ln lambda is 3.07276068510955
Sample  9 , trail 871 , rejecting H0
ln lambda is 3.0217694194872404
Sample  10 , trail 460 , rejecting H0
ln lambda is 3.0456088493159474


In [51]:
print('On average,', round(num/10),' iterations are needed to stop the test.')  

On average, 600  iterations are needed to stop the test.


As we can see from above results, it is possible to stop the test before going through all the samples.
One-sided test helps to check if the critical area of a distribution is either greater than or less 
than a certain value, but not both. 
Sequential A/B testing helped us to reduce the number samples required.
On average,600  iterations are needed to stop the test